# Metaheuristics TP7
Ning, Tientso

In [6]:
import numpy as np
import random
import math
from collections import deque

In [7]:
# This is the machine on which programs are executed
# The output is the value on top of the pile. 
class CPU:
    def __init__(self):
        self.pile= []
    def reset(self):
        while len(self.pile)>0:self.pile.pop()

In [18]:
# These are the instructions
def AND(cpu, data):
    try:
        v1 = cpu.pile.pop()
        v2 = cpu.pile.pop()
        cpu.pile.append(v1 and v2) #logical AND
    except:
        pass

def OR(cpu, data):
    try:
        v1 = cpu.pile.pop()
        v2 = cpu.pile.pop()
        cpu.pile.append(v1 or v2) #logical OR
    except:
        pass
    
def XOR(cpu, data):
    try:
        v1 = cpu.pile.pop()
        v2 = cpu.pile.pop()
        cpu.pile.append(v1 ^ v2) #logical XOR
    except:
        pass

def NOT(cpu, data):
    try:
        v = cpu.pile.pop()
        cpu.pile.append(not v) #logical NOT
    except:
        pass
    
# Push values of variables on the stack.      
def X1(cpu, data):
    cpu.pile.append(data[0])
def X2(cpu, data):
    cpu.pile.append(data[1])
def X3(cpu, data):
    cpu.pile.append(data[2])
def X4(cpu, data):
    cpu.pile.append(data[3])
    
# Execute a program
def execute(program,cpu, data):
    
    #check each instruction in our program
    for instruction in program:
        if instruction == "AND":
            AND(cpu, data)
        elif instruction == "OR":
            OR(cpu, data)
        elif instruction == "XOR":
            XOR(cpu, data)
        elif instruction == "NOT":
            NOT(cpu, data)
        elif instruction == "X1":
            X1(cpu, data)
        elif instruction == "X2":
            X2(cpu, data)
        elif instruction == "X3":
            X3(cpu, data)
        elif instruction == "X4":
            X4(cpu, data)
    try:
        return cpu.pile.pop() #top item in the stack is your answer
    except:
        return None

# Generate a random program
def randomProg(length,functionSet,terminalSet):
    
    program = []
    
    #generate a program of length length
    for i in range(length):
        
        #choose from functionSet or terminalSet, "randomly"
        if math.floor(random.random() + 0.5) < 1.0:
            program.append(functionSet[random.randint(0, len(functionSet)-1)])
        else:
            program.append(terminalSet[random.randint(0, len(terminalSet)-1)])

    return program

# Computes the fitness of a program. 
# The fitness counts how many instances of data in dataSet are correctly computed by the program
def computeFitness(prog,cpu,dataSet): 
    
    fitness = 0
    
    for i in range(0, len(dataSet)): #for each row of dataSet
        row = dataSet[i]
        a = execute(prog,cpu,row)
        if a == row[len(row)-1]: #last entry in row is the intended value
            fitness += 1
    
    return fitness

In [9]:
# Selection using 2-tournament.
def selection(Population,cpu,dataSet):
    listOfFitness=[]
    for i in range(len(Population)):
        prog=Population[i]
        f=computeFitness(prog,cpu,dataSet)
        listOfFitness.append( (i,f) )

    newPopulation=[]
    n=len(Population)
    for i in range(n):    
        i1=random.randint(0,n-1)
        i2=random.randint(0,n-1)
        if listOfFitness[i1][1]>listOfFitness[i2][1]:
            newPopulation.append(Population[i1])
        else:
            newPopulation.append(Population[i2])
    return newPopulation

def crossover(Population,p_c):
    newPopulation=[]
    n=len(Population)
    i=0
    while(i<n):
        p1=Population[i]
        p2=Population[(i+1)%n]
        m=len(p1)
        if random.random()<p_c:  # crossover
            k=random.randint(1,m-1)
            newP1=p1[0:k]+p2[k:m]
            newP2=p2[0:k]+p1[k:m]
            p1=newP1
            p2=newP2
        newPopulation.append(p1)
        newPopulation.append(p2)
        i+=2
    return newPopulation

def mutation(Population,p_m,terminalSet,functionSet):
    newPopulation=[]
    nT=len(terminalSet)-1
    nF=len(functionSet)-1
    for p in Population:
        for i in range(len(p)):
            if random.random()>p_m:continue
            if random.random()<0.5: 
                p[i]=terminalSet[random.randint(0,nT)]
            else:
                p[i]=functionSet[random.randint(0,nF)]
        newPopulation.append(p)
    return newPopulation

In [46]:
# LOOK-UP TABLE YOU HAVE TO REPRODUCE.
nbVar = 4
dataSet=[[0,0,0,0,0],[0,0,0,1,1],[0,0,1,0,0],[0,0,1,1,0],[0,1,0,0,0],[0,1,0,1,0],[0,1,1,0,0],[0,1,1,1,1],[1,0,0,0,0],[1,0,0,1,1],[1,0,1,0,0],[1,0,1,1,0],[1,1,0,0,0],[1,1,0,1,0],[1,1,1,0,0],[1,1,1,1,0]]
print(dataSet)

cpu=CPU()

# Function and terminal sets.
functionSet=["AND", "OR", "NOT", "XOR"]
terminalSet=["X1", "X2","X3", "X4"]

# Example of program.
prog=["X1", "X2", "AND", "X3", "OR"]
progLength = 5
prog=randomProg(progLength,functionSet,terminalSet)
print(prog)

# Execute a program on one row of the data set.
data = dataSet[0]
output=execute(prog,cpu,data)
print(output)
print("-------------")

# Parameters
popSize = 12
p_c = 0.8
p_m = 0.5

# Generate the initial population
population = deque()
fittest = deque()
for i in range(popSize):
    population.append(randomProg(progLength, functionSet, terminalSet)) #add programs

# Evolution. Loop on the creation of population at generation i+1 from population at generation i, through selection, crossover and mutation.
for i in range(50):
        
    population = selection(population, cpu, dataSet)
    population = crossover(population, p_c)
    population = mutation(population, p_m, terminalSet, functionSet)

    #keep best indi
    best_fit = 0.0
    for indi in population:
        potential = computeFitness(indi, cpu, dataSet)
        if potential >= best_fit:
            best = indi
    fittest.append(best)
    population.append(best)

top_val = [None, 0.0]
for p in population:
    if computeFitness(p, cpu, dataSet) > top_val[1]:
        top_val = [p, computeFitness(p, cpu, dataSet)]
print(top_val)

[[0, 0, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 0], [1, 0, 0, 1, 1], [1, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0]]
['XOR', 'X3', 'X3', 'AND', 'X3']
0
-------------
[['X3', 'XOR', 'X3', 'X3', 'XOR'], 13]


In [47]:
def find_optimal_setup (popSize, p_c, p_m, generations):
    nbVar = 4
    dataSet=[[0,0,0,0,0],[0,0,0,1,1],[0,0,1,0,0],[0,0,1,1,0],[0,1,0,0,0],[0,1,0,1,0],[0,1,1,0,0],[0,1,1,1,1],[1,0,0,0,0],[1,0,0,1,1],[1,0,1,0,0],[1,0,1,1,0],[1,1,0,0,0],[1,1,0,1,0],[1,1,1,0,0],[1,1,1,1,0]]
    # Function and terminal sets.
    functionSet=["AND", "OR", "NOT", "XOR"]
    terminalSet=["X1", "X2","X3", "X4"]
    
    fit_trials = []
    for trials in range(25):
        # Generate the initial population
        population = deque()
        fittest = deque()
        for i in range(popSize):
            population.append(randomProg(progLength, functionSet, terminalSet)) #add programs

        # Evolution. Loop on the creation of population at generation i+1 from population at generation i, through selection, crossover and mutation.
        for i in range(generations):

            population = selection(population, cpu, dataSet)
            population = crossover(population, p_c)
            population = mutation(population, p_m, terminalSet, functionSet)

            #keep best indi
            best_fit = 0.0
            for indi in population:
                potential = computeFitness(indi, cpu, dataSet)
                if potential >= best_fit:
                    best = indi
            fittest.append(best)
            population.append(best)
    
        #top-value
        top_val = [None, 0.0]
        for p in population:
            if computeFitness(p, cpu, dataSet) > top_val[1]:
                top_val = [p, computeFitness(p, cpu, dataSet)]
        fit_trials.append(top_val)
    
    for item in fit_trials:
        print(item[0], item[1])

In [43]:
for i in range (0, 10):
    for j in range (0, 10):
        find_optimal_setup(12, i*0.1, j*0.1, 50)

With Pc = 0.0, Pm = 0.0
Average: 9.48
With Pc = 0.0, Pm = 0.1
Average: 8.72
With Pc = 0.0, Pm = 0.2
Average: 7.48
With Pc = 0.0, Pm = 0.30000000000000004
Average: 7.88
With Pc = 0.0, Pm = 0.4
Average: 7.48
With Pc = 0.0, Pm = 0.5
Average: 7.08
With Pc = 0.0, Pm = 0.6000000000000001
Average: 6.64
With Pc = 0.0, Pm = 0.7000000000000001
Average: 7.2
With Pc = 0.0, Pm = 0.8
Average: 7.04
With Pc = 0.0, Pm = 0.9
Average: 7.64
With Pc = 0.1, Pm = 0.0
Average: 10.4
With Pc = 0.1, Pm = 0.1
Average: 12.2
With Pc = 0.1, Pm = 0.2
Average: 12.08
With Pc = 0.1, Pm = 0.30000000000000004
Average: 12.36
With Pc = 0.1, Pm = 0.4
Average: 11.92
With Pc = 0.1, Pm = 0.5
Average: 12.64
With Pc = 0.1, Pm = 0.6000000000000001
Average: 12.08
With Pc = 0.1, Pm = 0.7000000000000001
Average: 12.08
With Pc = 0.1, Pm = 0.8
Average: 12.32
With Pc = 0.1, Pm = 0.9
Average: 12.2
With Pc = 0.2, Pm = 0.0
Average: 9.12
With Pc = 0.2, Pm = 0.1
Average: 12.88
With Pc = 0.2, Pm = 0.2
Average: 12.6
With Pc = 0.2, Pm = 0.30000

In [54]:
find_optimal_setup(20,0.8,0.5, 250)

['XOR', 'AND', 'X3', 'NOT', 'AND'] 13
['AND', 'X4', 'X1', 'AND', 'AND'] 13
['AND', 'AND', 'X2', 'X2', 'XOR'] 13
['NOT', 'X4', 'X2', 'X2', 'XOR'] 13
['XOR', 'X3', 'X3', 'X3', 'XOR'] 13
['X3', 'X3', 'X2', 'X2', 'XOR'] 13
['AND', 'X3', 'X3', 'XOR', 'AND'] 13
['AND', 'X4', 'X2', 'OR', 'AND'] 15
['X3', 'NOT', 'X3', 'AND', 'AND'] 13
['X1', 'NOT', 'X1', 'X1', 'XOR'] 13
['NOT', 'XOR', 'X1', 'X1', 'XOR'] 13
['X3', 'NOT', 'X3', 'NOT', 'XOR'] 13
['AND', 'X2', 'X2', 'XOR', 'AND'] 13
['X1', 'AND', 'X4', 'X4', 'XOR'] 13
['XOR', 'X4', 'X2', 'NOT', 'AND'] 13
['X2', 'X4', 'NOT', 'AND', 'AND'] 13
['XOR', 'X1', 'XOR', 'X4', 'AND'] 13
['X3', 'X1', 'X1', 'XOR', 'AND'] 13
['X4', 'OR', 'X4', 'X4', 'XOR'] 13
['X3', 'X3', 'NOT', 'AND', 'AND'] 13
['X2', 'NOT', 'X2', 'XOR', 'NOT'] 13
['X4', 'XOR', 'AND', 'X2', 'AND'] 13
['OR', 'X1', 'NOT', 'AND', 'AND'] 13
['X4', 'X1', 'X1', 'NOT', 'AND'] 13
['X4', 'X4', 'XOR', 'X2', 'AND'] 13
